In [1]:
import os

In [2]:
%pwd

'c:\\Users\\rahul\\Facial_review_system\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\rahul\\Facial_review_system'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    base_model_path: Path
    

In [6]:
from Facial_Review_System.constants import *
from Facial_Review_System.utils.common import read_yaml, create_directories

In [7]:
#this class for base model configuration

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            base_model_path=Path(config.base_model_path)
        )

        return prepare_base_model_config

In [8]:
import os
import zipfile
import gdown
from Facial_Review_System import logger
from Facial_Review_System.utils.common import get_size

In [9]:
#this class is for downloading model 
class ModelDownload:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def download_file(self) -> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            os.makedirs(self.config.root_dir, exist_ok=True)  # Updated directory path
            logger.info(f"Downloading data from {dataset_url} into file {self.config.local_data_file}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix + file_id, self.config.local_data_file)  # Save to local_data_file

            logger.info(f"Downloaded data from {dataset_url} into file {self.config.local_data_file}")

        except Exception as e:
            raise e

In [10]:
try:
    config = ConfigurationManager()
    model_config = config.get_model_config() 
    vgg_model = ModelDownload(config = model_config)
    zip_download_dir = model_config.local_data_file 
    vgg_model.download_file()
except Exception as e:
    raise e

[2024-04-28 20:01:44,853: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-28 20:01:44,859: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-28 20:01:44,863: INFO: common: created directory at: artifacts]
[2024-04-28 20:01:44,866: INFO: common: created directory at: artifacts/prepare_base_model]
[2024-04-28 20:01:44,870: INFO: 4224232533: Downloading data from https://drive.google.com/file/d/10oVyfXZwo0o_QsYPfYyFOHF6145IDDvn/view?usp=drive_link into file artifacts/prepare_base_model/base_model.h5]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=10oVyfXZwo0o_QsYPfYyFOHF6145IDDvn
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=10oVyfXZwo0o_QsYPfYyFOHF6145IDDvn&confirm=t&uuid=1edb483b-644b-4ad8-a9eb-4a26fcdcb627
To: c:\Users\rahul\Facial_review_system\artifacts\prepare_base_model\base_model.h5
100%|██████████| 580M/580M [01:39<00:00, 5.81MB/s] 

[2024-04-28 20:03:27,270: INFO: 4224232533: Downloaded data from https://drive.google.com/file/d/10oVyfXZwo0o_QsYPfYyFOHF6145IDDvn/view?usp=drive_link into file artifacts/prepare_base_model/base_model.h5]
